In [2]:
import os
print(os.getcwd())  # In thư mục hiện tại
print(os.listdir(r"D:\DATN\QA_System"))  # Kiểm tra nội dung thư mục QA_System
print(os.listdir(r"D:\DATN\QA_System\code"))  # Kiểm tra nội dung thư mục code

d:\DATN\QA_System\eval
['.env', '.env.example', '.git', '.gitignore', 'code', 'data_analyze', 'data_hust', 'docker-compose.yml', 'docker_help.md', 'eval', 'img', 'README.md', 'requirements.txt', 'volumes']
['app.log', 'app.py', 'check_db.py', 'chunking.py', 'clutering.py', 'data_ingestion.py', 'final_df768.pkl', 'milvus_db.py', 'model_config.py', 'pipeline.ipynb', 'pipeline.py', 'raptor.ipynb', 'raptor.py', 'raptorpipeline.py', 'test.ipynb', 'utils.py', '__init__.py', '__pycache__']


In [5]:
import os
import sys
code_path = r"D:\DATN\QA_System\code"
sys.path.append(code_path)

# Import trực tiếp từ các file
from utils import convert_df_to_documents
from model_config import VietnameseEmbeddings, load_gemini2
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
def load_final_df():
    final_df_path = r"D:\DATN\QA_System\data_analyze\finaldf.pkl"
    if os.path.exists(final_df_path):
        final_df = pd.read_pickle(final_df_path)
        print(f"Đã load final_df từ file: {final_df_path}")
        return final_df
    else:
        print("File final_df chưa tồn tại.")
        return None



c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
final_df = load_final_df()
documents = convert_df_to_documents(final_df)

Đã load final_df từ file: D:\DATN\QA_System\data_analyze\finaldf.pkl
Processed 1198 out of 1198 documents.


In [7]:
embedding = VietnameseEmbeddings()
model = load_gemini2()

Initializing Vietnamese embedding model: keepitreal/vietnamese-sbert


c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
from ragas.llms import LangchainLLMWrapper 
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
#generator_llm = LangchainLLMWrapper(model)
generator_embeddings = LangchainEmbeddingsWrapper(embedding)

In [12]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(documents, testset_size=10)

Applying CustomNodeFilter:   5%|▍         | 58/1198 [00:03<00:50, 22.76it/s]Node a437378f-598f-4da6-b0f1-0dd411a60467 does not have a summary. Skipping filtering.
Node 99258e67-5344-4c28-b2fa-4b4bb986a02c does not have a summary. Skipping filtering.
Applying CustomNodeFilter:   5%|▌         | 61/1198 [00:03<00:56, 20.25it/s]Node b833f5bd-40bf-4f12-aff2-e39546a26baf does not have a summary. Skipping filtering.
Node 3b7c7bba-c243-41cb-afe8-11eb216fa900 does not have a summary. Skipping filtering.
Node 4fb7c4e9-c9a0-4e6a-ab17-97b2cb44c83b does not have a summary. Skipping filtering.
Node 54059ad1-dd81-4c36-ba77-e463aac023a3 does not have a summary. Skipping filtering.
Node faec07df-a769-40c5-b00b-31e9fbb1b7d4 does not have a summary. Skipping filtering.
Node bb4df28c-f6cc-43bc-8931-181d220cab6f does not have a summary. Skipping filtering.
Node 0f49873b-3cf1-47c2-84de-74f3974a4ec7 does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  86%|████████▌ | 1025/1198 [00:54<00:1

In [13]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Who is PGS.TS. Huỳnh Đăng Chính?,[Giám đốc Đại học: PGS.TS. Huỳnh Quyết Thắng\n...,PGS.TS. Huỳnh Đăng Chính is a Vice Director at...,single_hop_specifc_query_synthesizer
1,Could you provide detailed contact information...,[Cụm văn bản này cung cấp thông tin liên hệ củ...,The context provides the name and title of PGS...,single_hop_specifc_query_synthesizer
2,What are the development goals of Đại học Bách...,[CHIẾN LƯỢC PHÁT TRIỂN\nTÓM LƯỢC CHIẾN LƯỢC PH...,The development goals of Đại học Bách Khoa Hà ...,single_hop_specifc_query_synthesizer
3,Cach mang cong nghiep 4.0 la gi?,[Tóm tắt Chiến lược Phát triển Đại học Bách Kh...,Chiến lược phát triển của Đại học Bách Khoa Hà...,single_hop_specifc_query_synthesizer
4,what happen in 2020 at hanoi university?,[CHƯƠNG TRÌNH ĐÀO TẠO KỸ SƯ CHUYÊN SÂU ĐẶC THÙ...,"In 2020, Hanoi University of Science and Techn...",single_hop_specifc_query_synthesizer
5,Where can students find detailed instructions ...,[<1-hop>\n\nMật khẩu email: Liên hệ Trung tâm ...,Students can find detailed instructions for us...,multi_hop_specific_query_synthesizer
6,What role does the 'Hội đồng xét' play in the ...,[<1-hop>\n\n2. Phòng/Ban chức năng Đào tạo: Cu...,The 'Hội đồng xét' plays a crucial role in bot...,multi_hop_specific_query_synthesizer
7,Who are TS. Nguyễn An Hưng and TS. Nguyễn Hồng...,[<1-hop>\n\nCụm văn bản cung cấp thông tin về ...,TS. Nguyễn An Hưng is the Phó Bí thư Đoàn trườ...,multi_hop_specific_query_synthesizer
8,How Trường ĐH Bách khoa Hà Nội make sure their...,"[<1-hop>\n\nTheo đó, Trường ĐH Bách khoa Hà Nộ...",Trường ĐH Bách khoa Hà Nội ensures their speci...,multi_hop_specific_query_synthesizer
9,How do the regulations outlined in Quyết định ...,[<1-hop>\n\n3. Mức phí gửi xe: Mức phí gửi xe ...,The regulations outlined in Quyết định số 44/2...,multi_hop_specific_query_synthesizer


In [ ]:
#from datasets import load_dataset

# loading the V2 dataset
# amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")["eval"]
# amnesty_qa

In [ ]:
# RAGAS expects a file_name dict as key
for document in chunks:
    document.metadata['file_name'] = document.metadata['source']

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI

embedding = OpenAIEmbeddings()
model = ChatOpenAI()

vectorstore = Chroma.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context","question"]
  )

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

In [ ]:
from datasets import Dataset

questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

data = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

for query in questions:
    data["question"].append(query)
    data["answer"].append(rag_chain.invoke(query))
    data["contexts"].append([doc.page_content for doc in retriever.get_relevant_documents(query)])

dataset = Dataset.from_dict(data)

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_relevancy,
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

In [ ]:
result.to_pandas()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

df = result.to_pandas()

heatmap_data = df[['context_relevancy', 'context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']]

cmap = LinearSegmentedColormap.from_list('green_red', ['red', 'green'])

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, annot=True, fmt=".2f", linewidths=.5, cmap=cmap)

plt.yticks(ticks=range(len(df['question'])), labels=df['question'], rotation=0)

plt.show()


### Add LangFuse

In [ ]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-8be80c67-4187-4e43-9d01-544195dc9f03",
  public_key="pk-lf-d7653f64-8086-4365-b05c-865ead3478a3",
  host="http://localhost:3000"
)

In [ ]:
trace = langfuse.trace(
    name = "eval",
    user_id = "eval_user",
    metadata = {
        "email": "prod@company.com",
    },
    tags = ["evaluation"]
)

In [ ]:
df = result.to_pandas()

In [ ]:
for _, row in df.iterrows():
    for metric_name in ["faithfulness", "answer_relevancy", "context_recall"]:
        langfuse.score(
            name=metric_name,
            value=row[metric_name],
            trace_id=trace.id
        )